In [1]:
%matplotlib inline
import pandas as pd
import json
import altair as alt
from vega_datasets import data
import matplotlib.pyplot as plt
pd.__version__

'1.4.4'

In [2]:
# all national parks
# https://github.com/joshuakemmerling/AwesomeData/blob/master/national-parks.csv
parks = pd.read_csv('data/national-parks.csv')
parks['park'] = parks['name'].apply(lambda s:' '.join(s.replace('National', '').replace('Park', '').strip().split(" ")))
parks.head()

,name,location,park
0,Acadia National Park,Maine,Acadia
1,American Samoa National Park,American Samoa Territory,American Samoa
2,Arches National Park,Utah,Arches
3,Badlands National Park,South Dakota,Badlands
4,Big Bend National Park,Texas,Big Bend


In [3]:
# data with national parks lat/long
# https://github.com/sughodke/D3-US-Graph/blob/master/nationalparks.csv
parks_long = pd.read_csv('data/nationalparks.csv')
parks_long['details'] = parks_long['details'].apply(lambda s:s.replace('"', "").strip())
parks_long['name'] = parks_long['details'].apply(lambda s: ' '.join(s.split(" ")[2:]))
parks_long.head()

,longitude,latitude,details,name
0,-154.88689,58.58305,USA-National Park Katmai,Katmai
1,-116.00890,33.95290,USA-National Park Joshua Tree,Joshua Tree
2,-88.89170,47.96240,USA-National Park Isle Royale,Isle Royale
3,-93.02210,34.37960,USA-National Park Hot Springs,Hot Springs
4,-155.30000,19.40000,USA-National Park Hawaii Volcanoes,Hawaii Volcanoes


In [71]:
# set the longitude and latitude of parks from parks_long
parks['longitude'] = -1
parks['latitude'] = -1
for i, park in parks.iterrows():
    if parks_long['details'].str.contains(park.park).any():
        index = parks_long[parks_long['name'].str.match(park.park)].index[0]
        row = parks_long.loc[index]
        parks.loc[i, ['longitude', 'latitude']] = [row.longitude, row.latitude]
        
# hacky way to add coordinates to parks with missing coordinates
# they had Glacier wrong
coordinates = [['Glacier', -113.7870, 48.7596],
               ['American Samoa', -170.68333, -14.25833],
               ['Gateway Arch', -90.184776, 38.624691],
               ['Indiana Dunes', -87.053762, 41.633349],
               ['New River Gorge', -81.042973, 37.896731],
               ['Pinnacles', -121.197243, 36.491508],
               ['Virgin Islands', -64.7685, 18.3515],
               ['White Sands', -106.171669, 32.779720]]

for coordinate in coordinates:
    parks.loc[parks.park == coordinate[0], ['longitude', 'latitude']] = coordinate[1:]

#parks.to_csv('parks.csv')    

### working exploratory viz

In [74]:
states = alt.topo_feature(data.us_10m.url, feature='states')

background = alt.Chart(
    states,
    title=alt.Title("U.S. National Parks", fontSize=18)
).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=800,
    height=500
)

points = alt.Chart(parks).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.value(40),
    tooltip=[
        alt.Tooltip('name', title='Name'), 
        alt.Tooltip('location', title='State'),
        alt.Tooltip('longitude:Q', title='Longitude'), 
        alt.Tooltip('latitude:Q', title='Latitude')
    ]
)

(background + points).interactive()

alt.LayerChart(...)

### viz with some bar chart interaction

In [93]:
states = alt.topo_feature(data.us_10m.url, feature='states')

background = alt.Chart(
    states,
    title=alt.Title("U.S. National Parks", fontSize=18)
).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=600,
    height=400
)

highlight = alt.selection_single(on='click', fields=['name'], empty='none')

points = alt.Chart(parks).mark_circle().encode(
    color=alt.condition(highlight, alt.value('pink'), alt.value('steelblue')),
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.value(60),
    tooltip=[
        alt.Tooltip('name', title='Name'), 
        alt.Tooltip('location', title='State'),
        alt.Tooltip('longitude:Q', title='Longitude'), 
        alt.Tooltip('latitude:Q', title='Latitude')
    ]
).add_selection(
    highlight
)

bars = alt.Chart(parks).mark_bar().encode(
    x='longitude',
    y='latitude'
).transform_filter(
    highlight
)

(background + points) | bars

alt.HConcatChart(...)

### viz with some hover over state interaction

In [91]:
highlight = alt.selection_single(on='mouseover', fields=['id'], empty='none')

background = alt.Chart(
    states,
    title=alt.Title("U.S. National Parks", fontSize=18)
).mark_geoshape().encode(
    color=alt.condition(highlight, alt.value('red'), 'id:Q'),
    tooltip=['id:Q']
).add_selection(highlight).project(
    type='albersUsa'
).properties(
    width=900,
    height=600
)

points = alt.Chart(parks).mark_circle().encode(
    color=alt.condition(highlight, alt.value('red'), alt.value('blue')),
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.value(40),
    tooltip=[
        alt.Tooltip('name', title='Name'), 
        alt.Tooltip('location', title='State'),
        alt.Tooltip('longitude', title='Longitude'), 
        alt.Tooltip('latitude', title='Latitude')
    ]
).add_params(highlight)


(background + points)

alt.LayerChart(...)